In [ ]:
import os
import pandas as pd
from tkinter import Tk, Label, OptionMenu, StringVar, Button, Canvas, Scrollbar, Frame
from PIL import Image, ImageTk

1. What are the grains you're going to use? ID them with the index by the SAM model
2. What's the type of each grain?
3. Rename the grain with GrainType_SampleID_index

In [ ]:
from save_selections_to_excel import save_selections_to_excel

Update the index_list and excel_file_path for the sample you are going to label

In [ ]:
index_list_keep = [70, 78, 48, 34, 31, 23, 43, 85, 56, 39, 30, 14, 35, 10, 44, 69, 83, 57, 90, 75, 11, 4, 18, 9, 5, 32, 25, 29, 12, 38, 49, 81, 87, 80, 37, 36, 66, 58, 60, 59, 15, 17, 46, 21, 53, 62, 45, 41, 42, 63, 1, 7, 76, 19, 3, 13, 16, 40, 47, 20, 79, 27, 64, 72]
excel_file_path = r"Seg_Images\USU-4183B 150-250 Elemental Map\USU-4183B 150-250 Elemental Map_indexes.xlsx"
image_folder_path = r'Seg_Images\USU-4183B 150-250 Elemental Map'

In [ ]:
# index_list_keep = [25, 44, 28, 11, 43, 35, 12, 32, 1, 42, 3, 24, 7, 38, 21, 37, 45, 33, 22, 15, 41, 54, 31, 30, 40, 23, 20, 29, 6, 16, 8, 18, 5, 2, 10, 34, 4, 39, 0, 27, 14, 13, 9, 17]
# excel_file_path = r"Seg_Images\USU-4169B 250-355 Elemental Map_aligned\USU-4169B 250-355 Elemental Map_aligned_indexes.xlsx"
# image_folder_path = r'Seg_Images\USU-4169B 250-355 Elemental Map_aligned'

In [ ]:
save_selections_to_excel(index_list_keep, excel_file_path)

**Will you also extract a corresponded part of K-map? If so, jump to Option #2**

# Resize to 224*224, Label and Rename

In [ ]:
import os
import pandas as pd
from PIL import Image, ImageOps
import numpy as np

def crop_center_non_black_strict(image_path, output_size=224):
    """
    Crop or resize the image to ensure the object is centered and the final image size is 224x224.
    If necessary, shrink the image while maintaining the aspect ratio and then pad to ensure the final size.
    """
    # Load image
    image = Image.open(image_path)
    
    # Convert image to numpy array
    img_array = np.array(image)
    
    # Identify non-black pixels
    non_black_pixels = np.any(img_array[:, :, :3] != 0, axis=-1)
    
    # Find the bounding box of non-black pixels
    non_black_coords = np.argwhere(non_black_pixels)
    if non_black_coords.size == 0:
        # If no non-black pixels found, return the original image resized to 224x224
        image = image.resize((output_size, output_size), Image.Resampling.LANCZOS)
        return image
    
    # Bounding box of the non-black pixels
    top_left = non_black_coords.min(axis=0)
    bottom_right = non_black_coords.max(axis=0)
    object_height = bottom_right[0] - top_left[0]
    object_width = bottom_right[1] - top_left[1]

    # Crop the image to the bounding box
    cropped_img_array = img_array[top_left[0]:bottom_right[0], top_left[1]:bottom_right[1]]
    cropped_image = Image.fromarray(cropped_img_array)

    # Determine the new size while maintaining aspect ratio
    object_ratio = object_width / object_height
    if object_ratio > 1:
        # Object is wider than tall, resize based on width
        new_width = output_size
        new_height = int(new_width / object_ratio)
    else:
        # Object is taller than wide, resize based on height
        new_height = output_size
        new_width = int(new_height * object_ratio)

    resized_image = cropped_image.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # Calculate padding to center the object in a 224x224 frame
    delta_w = output_size - new_width
    delta_h = output_size - new_height
    padding = (delta_w // 2, delta_h // 2, delta_w - (delta_w // 2), delta_h - (delta_h // 2))

    # Pad the image to ensure it is 224x224 and the object is centered
    padded_image = ImageOps.expand(resized_image, padding, fill=(0, 0, 0))
    
    return padded_image

def resize_and_save_images(input_directory, excel_file_path):
    """
    Read an Excel file to get a list of images to process based on the 'keep' value.
    Only images with 'keep' value of 1 will be cropped and resized to 224x224 and saved in a 'Selected_and_Resized' folder.
    """
    # Check if input directory exists
    if not os.path.exists(input_directory):
        print(f"Directory '{input_directory}' does not exist.")
        return

    # Create the 'Selected_and_Resized' output folder under the input directory
    output_directory = os.path.join(input_directory, 'Selected_and_Resized')
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Created output directory: {output_directory}")

    # Read the Excel file and filter images with keep == 1
    df = pd.read_excel(excel_file_path)
    if 'keep' not in df.columns:
        print("The Excel file doesn't have a 'keep' column.")
        return

    # Filter the indexes (image names without extensions) where keep == 1
    images_to_keep = df[df['keep'] == 1]['Index'].astype(str).tolist()

    # Process each file in the input directory
    for file_name in os.listdir(input_directory):
        file_name_no_ext = os.path.splitext(file_name)[0]  # Get file name without extension
        if file_name_no_ext in images_to_keep:
            file_path = os.path.join(input_directory, file_name)
            if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Crop and resize image
                cropped_image = crop_center_non_black_strict(file_path)
                
                # Save cropped image to the 'Selected_and_Resized' folder
                cropped_file_path = os.path.join(output_directory, file_name)
                cropped_image.save(cropped_file_path)
                print(f"Resized composite map grains: {cropped_file_path}")


In [ ]:
resize_and_save_images(image_folder_path, excel_file_path)

# Option #2 (more prefered): withdraw corresponded part of the K map, and resize and crop together with the composite elementary map

In [ ]:
import os
import pandas as pd
from PIL import Image, ImageOps
import numpy as np
from load_masks_from_json import load_masks_from_json

def crop_center_non_black_strict(image_input, output_size=224):
    """
    Crop or resize the image to ensure the object is centered and the final image size is 224x224.
    If necessary, shrink the image while maintaining the aspect ratio and then pad to ensure the final size.
    Accepts either a file path or a PIL image object.
    """
    # Load the image if a file path is provided
    if isinstance(image_input, str):
        image = Image.open(image_input)
    else:
        image = image_input  # Assume it's a PIL Image object
    
    # Convert image to numpy array
    img_array = np.array(image)
    
    # Identify non-black pixels
    non_black_pixels = np.any(img_array[:, :, :3] != 0, axis=-1)
    
    # Find the bounding box of non-black pixels
    non_black_coords = np.argwhere(non_black_pixels)
    if non_black_coords.size == 0:
        # If no non-black pixels found, return the original image resized to 224x224
        image = image.resize((output_size, output_size), Image.Resampling.LANCZOS)
        return image
    
    # Bounding box of the non-black pixels
    top_left = non_black_coords.min(axis=0)
    bottom_right = non_black_coords.max(axis=0)
    object_height = bottom_right[0] - top_left[0]
    object_width = bottom_right[1] - top_left[1]

    # Crop the image to the bounding box
    cropped_img_array = img_array[top_left[0]:bottom_right[0], top_left[1]:bottom_right[1]]
    cropped_image = Image.fromarray(cropped_img_array)

    # Determine the new size while maintaining aspect ratio
    object_ratio = object_width / object_height
    if object_ratio > 1:
        # Object is wider than tall, resize based on width
        new_width = output_size
        new_height = int(new_width / object_ratio)
    else:
        # Object is taller than wide, resize based on height
        new_height = output_size
        new_width = int(new_height * object_ratio)

    resized_image = cropped_image.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # Calculate padding to center the object in a 224x224 frame
    delta_w = output_size - new_width
    delta_h = output_size - new_height
    padding = (delta_w // 2, delta_h // 2, delta_w - (delta_w // 2), delta_h - (delta_h // 2))

    # Pad the image to ensure it is 224x224 and the object is centered
    padded_image = ImageOps.expand(resized_image, padding, fill=(0, 0, 0))
    
    return padded_image


def apply_mask_to_k_map(mask_list, k_map, index):
    """
    Apply the mask corresponding to the index from a list of masks to the K_map image.
    The region outside the mask will be blacked out.
    """
    # Ensure index is within the bounds of the mask list
    if index < 0 or index >= len(mask_list):
        print(f"No mask found for index {index}")
        return None
    
    # Get the binary mask (numpy array) for the given index
    mask_index = mask_list[index]['segmentation']  # Use the 'segmentation' field
    
    # Convert K_map to a numpy array
    k_map_array = np.array(k_map)
    
    # Ensure the mask has the same shape as the K_map
    if mask_index.shape != k_map_array.shape[:2]:  # Assuming K_map is a color image
        print(f"Mask shape {mask_index.shape} does not match K_map shape {k_map_array.shape[:2]}")
        return None
    
    # Apply the mask to the K_map (keep only the masked region, rest will be black)
    masked_k_map = np.zeros_like(k_map_array)
    masked_k_map[mask_index > 0] = k_map_array[mask_index > 0]
    
    return Image.fromarray(masked_k_map)


def resize_and_save_images_with_k_map(input_directory, excel_file_path, mask, k_map_path):
    """
    Read an Excel file to get a list of images to process based on the 'keep' value.
    Only images with 'keep' value of 1 will be cropped and resized to 224x224 and saved in a 'Selected_and_Resized' folder.
    Additionally, for each image, a corresponding temporary K_map image will be generated, processed, and saved.
    """
    # Check if input directory exists
    if not os.path.exists(input_directory):
        print(f"Directory '{input_directory}' does not exist.")
        return

    # Create the 'Selected_and_Resized' output folder under the input directory
    output_directory = os.path.join(input_directory, 'Selected_and_Resized')
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Created output directory: {output_directory}")

    # Create the 'Selected_and_Resized_K' output folder under the input directory
    output_directory_k = os.path.join(input_directory, 'Selected_and_Resized_K')
    if not os.path.exists(output_directory_k):
        os.makedirs(output_directory_k)
        print(f"Created output directory for K_map images: {output_directory_k}")

    # Read the Excel file and filter images with keep == 1
    df = pd.read_excel(excel_file_path)
    if 'keep' not in df.columns:
        print("The Excel file doesn't have a 'keep' column.")
        return

    # Filter the indexes (image names without extensions) where keep == 1
    images_to_keep = df[df['keep'] == 1]['Index'].astype(str).tolist()

    # Load the K_map
    k_map = Image.open(k_map_path)

    # Process each file in the input directory
    for file_name in os.listdir(input_directory):
        file_name_no_ext = os.path.splitext(file_name)[0]  # Get file name without extension
        if file_name_no_ext in images_to_keep:
            file_path = os.path.join(input_directory, file_name)
            if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Crop and resize the original image
                cropped_image = crop_center_non_black_strict(file_path)
                
                # Save cropped image to the 'Selected_and_Resized' folder
                cropped_file_path = os.path.join(output_directory, file_name)
                cropped_image.save(cropped_file_path)
                print(f"Resized Composite map grains: {cropped_file_path}")

                # Apply mask to the K_map for the current index
                index = int(file_name_no_ext)  # Assuming index is an integer in the file name
                masked_k_map_image = apply_mask_to_k_map(mask, k_map, index)
                
                if masked_k_map_image:
                    # Resize and crop the masked K_map image the same way
                    resized_k_map_image = crop_center_non_black_strict(masked_k_map_image)

                    # Save the masked and resized K_map image with a "_K" suffix in the 'Selected_and_Resized_K' folder
                    k_map_file_name = f"{file_name_no_ext}_K.png"
                    k_map_file_path = os.path.join(output_directory_k, k_map_file_name)
                    resized_k_map_image.save(k_map_file_path)
                    print(f"Resized Corresponded K map grains: {k_map_file_path}")

In [ ]:
# excel_file_path = r'Seg_Images\USU-4183B 250-355 Elemental Map_aligned\USU-4183B 250-355 Elemental Map_aligned_indexes.xlsx'
# folder_path = r'Seg_Images\USU-4183B 250-355 Elemental Map_aligned'
# k_map_path = r'1_Raw_Data\Segmented\USU-4183B\SEM Graphs\USU-4183B 250-355 K Map_aligned.png'
# mask_path = r'Seg_Images\USU-4183B 250-355 Elemental Map_aligned\USU-4183B 250-355 Elemental Map_aligned.json'
k_map_path = r"Aligned\USU-4169B 250-355 Elemental Map\USU-4169B 250-355 K Map_aligned.png"
mask_path =   r'Seg_Images\USU-4169B 250-355 Elemental Map_aligned\USU-4169B 250-355 Elemental Map_aligned.json'

In [ ]:
mask = load_masks_from_json(mask_path)

In [ ]:
resize_and_save_images_with_k_map(input_directory=image_folder_path, excel_file_path=excel_file_path, mask=mask, k_map_path=k_map_path)